In [1]:
from GetData.GetTimeSeries import GetTimeSeries
from SQLite.SQLite import SQLite
from datetime import date
from tqdm import tqdm

import pandas as pd

In [2]:
def create_table():
    for query in ['create_ts.sql', 'create_meta.sql']:
        with open(f'SQL/{query}', 'r') as f:
            query = f.read()
        SQL = SQLite('data.db')
        SQL(query)
create_table()

In [3]:
start_date = '01/01/2019'
end_date = '01/01/2022'
ids = pd.read_csv('time_series.csv').values.reshape(-1).tolist()
ids[0:10]

[1, 11, 12, 23, 27, 29, 31, 33, 35, 37]

In [4]:
ids[19]

226

In [5]:
data_t = 'time_series'
meta_t = 'time_series_meta'
SQL = SQLite('data.db')
SQL(f'DELETE FROM {data_t}')
SQL(f'DELETE FROM {meta_t}')

GTS = GetTimeSeries(ids, start_date=start_date, end_date=end_date)
i = iter(GTS)
failed_list = []
for df, meta in tqdm(i):
    if df is None:
        break
    df.columns = ['value']
    df['series_id'] = meta.loc[0, 'code']
    df.index.names = ['d']
    df = df.reset_index()
    
    try:
        with SQL.get_connection() as con:
            df.to_sql(data_t, con, if_exists='append', index=False)
            meta.to_sql(meta_t, con, if_exists='append', index=False)
    except Exception as e:
        message = f"{meta.loc[0, 'code']}^{str(e)}"
        print(message)
        failed_list.append(message)

if len(failed_list) > 0:
    s = '\n'.join(failed_list)
    with open('failed_to_upload.txt', 'w') as f:
        f.write(s)

 10%|██████████                                                                                       | 20/194 [00:27<03:59,  1.38s/it]

226^UNIQUE constraint failed: time_series.d, time_series.series_id


 11%|██████████▌                                                                                      | 21/194 [00:30<04:41,  1.63s/it]

227^UNIQUE constraint failed: time_series.d, time_series.series_id


100%|████████████████████████████████████████████████████████████████████████████████████████████████| 194/194 [02:49<00:00,  1.15it/s]


In [6]:
print('done')

done
